In [128]:
import streamlit as st
import json
import pandas as pd
from pandas import json_normalize
import requests
# from tabulate import tabulate
from sklearn.cluster import KMeans
# import random
import numpy as np
import pandas as pd
import folium
from streamlit_folium import st_folium,folium_static
import warnings
warnings.filterwarnings('ignore')
import os
os.environ.keys()

KeysView(environ({'COMMAND_MODE': 'unix2003', 'CONDA_DEFAULT_ENV': 'base', 'CONDA_EXE': '/opt/anaconda3/bin/conda', 'CONDA_PREFIX': '/opt/anaconda3', 'CONDA_PROMPT_MODIFIER': '(base) ', 'CONDA_PYTHON_EXE': '/opt/anaconda3/bin/python', 'CONDA_SHLVL': '1', 'HOME': '/Users/MayankGarg', 'HOMEBREW_CELLAR': '/opt/homebrew/Cellar', 'HOMEBREW_PREFIX': '/opt/homebrew', 'HOMEBREW_REPOSITORY': '/opt/homebrew', 'INFOPATH': '/opt/homebrew/share/info:', 'LOGNAME': 'MayankGarg', 'LaunchInstanceID': 'A81EC171-ADED-45CB-8F5B-40E3D1D85A02', 'MANPATH': '/opt/homebrew/share/man::', 'MallocNanoZone': '0', 'MallocSpaceEfficient': '1', 'OLDPWD': '/', 'ORIGINAL_XDG_CURRENT_DESKTOP': 'undefined', 'PATH': '/opt/anaconda3/bin:/usr/local/opt/llvm/bin:/opt/anaconda3/bin:/opt/anaconda3/condabin:/opt/homebrew/bin:/opt/homebrew/sbin:/Library/Frameworks/Python.framework/Versions/3.10/bin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/Apple/usr/bin:/var/run/com.apple.security.crypte

In [129]:
location_name='DTU'
location_latitude='28.75005'
location_longitude='77.1156389'
radius=100000
thing='pg'
facilities_list_name=['Gym', 'Restaurents', 'cafe','market']

In [130]:

def fill_facilitites(search_query,List,latitude,longitude,ind):
    radius = '1000'
    url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(
        latitude, longitude, radius, search_query)
    results = requests.get(url).json()
    venues = json_normalize(results['items'])
    if len(venues) >0:
        List[ind].append(venues['title'].count())
    else:
        List[ind].append(0)

In [131]:
# search_query='gym'


# radius = '1000'
# url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{},{};r={}&q={}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(
#         latitude, longitude, radius, search_query)
# results = requests.get(url).json()
# venues = json_normalize(results['items'])
# List[ind].append(venues['title'].count())

In [132]:
def find_facilities(df_final,d2):
    latitudes = list(d2['position.lat'])
    longitudes = list(d2['position.lng'])
    List = [[] for _ in range(len(facilities_list_name))]
    for lat, lng in zip(latitudes, longitudes):
        latitude = lat
        longitude = lng


        for i in range(0,len(facilities_list_name)):
            # print(i)
            fill_facilitites(facilities_list_name[i], List, latitude, longitude, i)


    for i in range(0,len(List)):
        # st.sidebar.write(facilities_list_name[i] , " | ", len(List[i]) )
        df_final[facilities_list_name[i]]=List[i]


In [144]:

def make_cluster(df_final):
    if len(df_final) <=3 :
        df_final['Cluster']='0'
        return;
    # Run K-means clustering on dataframe
    kclusters = 3

    kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_final)
    df_final['Cluster'] = kmeans.labels_
    df_final['Cluster'] = df_final['Cluster'].apply(str)

In [145]:
def add_to_map(df_final,d2):
    # Plotting clustered locations on map using Folium

     # define coordinates of the college
    map_bom = folium.Map(location=[location_latitude, location_longitude], zoom_start=12)

    # instantiate a feature group for the incidents in the dataframe
    locations = folium.map.FeatureGroup()

    # set color scheme for the clusters
    def color_producer(cluster):
        if cluster == '0':
            return 'green'
        elif cluster == '1':
            return 'orange'
        else:
            return 'red'

    latitudes = list(df_final['position.lat'])
    longitudes = list(df_final['position.lng'])
    labels = list(df_final['Cluster'])
    names = list(d2['title'])
    for lat, lng, label, names in zip(latitudes, longitudes, labels, names):
        folium.CircleMarker(
            [lat, lng],
            fill=True,
                fill_opacity=1,
                popup=folium.Popup(names, max_width=300),
                radius=5,
                color=color_producer(label)
            ).add_to(map_bom)

        # add locations to map
    map_bom.add_child(locations)
    folium.Marker([location_latitude, location_longitude], popup=location_name).add_to(map_bom)
    return map_bom

In [146]:
# def draw_graph(location_name,location_latitude,location_longitude,radius):
#     url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{0},{1};r={2}&q={3}&apiKey=kW5ZzpawjBDJ0waMxzGx_a3cFfR9bgWK-5ejXI9xt1s'.format(
#             location_latitude, location_longitude, radius,thing)

#     # url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{0},{1};r={2}&q={3}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(
#     #     location_latitude, location_longitude, radius,thing)
#     data = requests.get(url).json()
#     d = json_normalize(data['items'])
#     if d.shape[0] ==0:
#         return -1,-1,0

#     d2 = d[['title','position.lat', 'position.lng']]

    


#     # Counting no. of cafes, department stores and gyms near apartments around IIT Bombay
#     df_final = d2[['position.lat', 'position.lng']]

#     find_facilities(df_final,d2)

#     # st.sidebar.write('after find_facilities')

#     make_cluster(df_final)

#     # create map object and plot the points

#     map_obj = add_to_map(df_final,d2)
#     name_df=pd.DataFrame()
#     name_df['name']=d2['title']
#     sz=len(df_final.columns)



#     for i in range(0,sz):
#         name_df[df_final.columns[i]]=df_final[df_final.columns[i]]

#     # df_final['name']=d2['title']



#     return map_obj, name_df,0

In [147]:

url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{0},{1};r={2}&q={3}&apiKey=kW5ZzpawjBDJ0waMxzGx_a3cFfR9bgWK-5ejXI9xt1s'.format(
            location_latitude, location_longitude, radius,thing)

    # url = 'https://discover.search.hereapi.com/v1/discover?in=circle:{0},{1};r={2}&q={3}&apiKey=uJHMEjeagmFGldXp661-pDMf4R-PxvWIu7I68UjYC5Q'.format(
    #     location_latitude, location_longitude, radius,thing)
data = requests.get(url).json()
d = json_normalize(data['items'])
    

d2 = d[['title','position.lat', 'position.lng']]

    


    # Counting no. of cafes, department stores and gyms near apartments around IIT Bombay
df_final = d2[['position.lat', 'position.lng']]

find_facilities(df_final,d2)

    # st.sidebar.write('after find_facilities')

make_cluster(df_final)

    # create map object and plot the points

map_obj = add_to_map(df_final,d2)
name_df=pd.DataFrame()
name_df['name']=d2['title']
sz=len(df_final.columns)



for i in range(0,sz):
    name_df[df_final.columns[i]]=df_final[df_final.columns[i]]

    # df_final['name']=d2['title']





In [148]:
df_final

,position.lat,position.lng,Gym,Restaurents,cafe,market,Cluster
0,28.75055,77.11646,0,20,1,19,0


In [149]:
d2

,title,position.lat,position.lng
0,Sakshi PG,28.75055,77.11646


In [136]:

map_obj, name_df,val =draw_graph(location_name,location_latitude,location_longitude,radius)
if val == 0:
    print('not available')
else:
    # folium_static(map_obj, width=700)
        # st.write(df_final)
        # st.write(name_df)
    thing=thing.upper()


    new_df_green=name_df[name_df['Cluster']=='0']
    new_df_green.drop(columns=['Cluster'],inplace=True)



    new_df_orange=name_df[name_df['Cluster']=='1']
    new_df_orange.drop(columns=['Cluster'],inplace=True)



    new_df_red=name_df[name_df['Cluster']=='2']
    new_df_red.drop(columns=['Cluster'],inplace=True)



ValueError: n_samples=1 should be >= n_clusters=3.

In [ ]:
new_df_green

,name,position.lat,position.lng,Gym,Restaurents,cafe,market
8,Anand Niketan-Gate No 5,28.58164,77.16392,8,20,18,20
9,Anand Niketan-Gate No 6,28.58061,77.16366,7,20,15,20
10,CPWD Block G Type 3-Main Gate,28.58130,77.16543,8,20,19,20
12,Air India Housing Colony-Gate No 2,28.55816,77.15732,10,20,14,19


In [ ]:
new_df_orange

,name,position.lat,position.lng,Gym,Restaurents,cafe,market
4,Vasant Apartment,28.57410,77.15650,3,20,1,16
5,Block E Vasant Vihar-Gate No 1,28.56455,77.15574,7,20,3,20
6,Air India Housing Colony-Gate No 1,28.55915,77.15333,4,20,3,13
7,Rbi Officer Flats,28.55974,77.15401,8,20,5,14
14,Shanti Niketan Gate No 11,28.57840,77.17260,3,20,8,20
15,Central Govt Employees Flats,28.57849,77.17274,3,20,8,20
16,Apartment Rental-Flat Rental,28.57372,77.17205,3,20,7,20


In [ ]:
new_df_red

,name,position.lat,position.lng,Gym,Restaurents,cafe,market
0,Shivam Properties,28.62039,77.04480,20,20,1,20
1,Kohinoor Guest House-Vikaspuri,28.63851,77.07318,20,20,12,20
2,Stay Royal Bed N Breakfast,28.62092,77.10485,17,20,4,20
3,Green PG,28.51015,77.05069,17,20,8,20
11,Block B Vasant Vihar-Gate No 2,28.56797,77.16053,12,20,4,20
13,Deer Park Home Stay,28.56604,77.16161,15,20,12,20
17,Narmada Apartment N1,28.52558,77.14411,11,20,7,20
18,Narmada Apartment N3,28.52537,77.14396,11,20,7,20
19,Narmada Apartment N9,28.52555,77.14422,11,20,7,20


In [ ]:
map_obj

,title,id,language,ontologyId,resultType,access,distance,categories,address.label,address.countryCode,...,address.county,address.city,address.district,address.subdistrict,address.postalCode,position.lat,position.lng,contacts,address.street,references
0,Kanishka Apartments,here:pds:place:356ttng7-96a3e6543a8878e1e6dc33...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.72976, 'lng': 77.15062}]",4099,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Kanishka Apartments, Block CD-Pocket 1, Shalim...",IND,...,North West,Delhi,Shalimar Bagh,Block CD-Pocket 1,110088,28.72978,77.15075,NaN,NaN,NaN
1,Tower Apartment,here:pds:place:356ttng7-cd3564b7536b4c40883144...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.69667, 'lng': 77.14261}]",6482,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Tower Apartment, Nishant Kunj, Pitampura, Delh...",IND,...,North West,Delhi,Pitampura,Nishant Kunj,110034,28.69680,77.14268,NaN,NaN,NaN
2,Baba Guest House,here:pds:place:356ttngk-c82fab4131db4df8a51925...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.7134, 'lng': 77.17506}]",7069,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Baba Guest House, Mandi Marg, Kewal Park Exten...",IND,...,North West,Delhi,Azadpur,Kewal Park Extension-Block A,110033,28.71351,77.17497,[{'phone': [{'value': '+918278196391'}]}],Mandi Marg,NaN
3,Ganesh Apartment,here:pds:place:356ttngs-703c3f0fe9ef4b508cf587...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.7438, 'lng': 77.20371}]",8624,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Ganesh Apartment, Sant Nagar-B Block, Burari, ...",IND,...,North,Delhi,Burari,Sant Nagar-B Block,110084,28.74384,77.20382,NaN,NaN,NaN
4,Narang Boys Pg,here:pds:place:356ttngh-d8e6c1e9bf4503b863458f...,en-GB,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.69572, 'lng': 77.2041}]",10525,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Narang Boys Pg, Om Mandir Marg, New Police Lin...",IND,...,North West,Delhi,Guru Teg Bahadur Nagar,New Police Lines,110009,28.69576,77.20406,NaN,Om Mandir Marg,NaN
5,Kohinoor Guest House-Vikaspuri,here:pds:place:356ttnfc-13a79363883a41eb9a57d7...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.63858, 'lng': 77.07318}]",13076,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Kohinoor Guest House-Vikaspuri, Guru Virjanand...",IND,...,West,Delhi,Vikas Puri,Block C,110018,28.63851,77.07318,NaN,Guru Virjanand Marg,NaN
6,Rooms on Rent,here:pds:place:356ttnfu-12cbab0939a746589fad15...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.64144, 'lng': 77.18536}]",13874,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Rooms on Rent, Satpal Bhatia Marg, Block 58, O...",IND,...,Central,Delhi,Old Rajendra Nagar,Block 58,110060,28.64136,77.18549,NaN,Satpal Bhatia Marg,NaN
7,Sadar Guest House,here:pds:place:356ttngh-7261f6f64986d8a88b99d8...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.6544, 'lng': 77.21135}]",14139,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Sadar Guest House, Dungarmal Surana Marg, Bast...",IND,...,Central,Delhi,Sadar Bazar,Basti Harphool Singh,110006,28.65444,77.21121,[{'phone': [{'value': '+911123675451'}]}],Dungarmal Surana Marg,NaN
8,Stay Royal Bed N Breakfast,here:pds:place:356aabd1-9c7b9cf86e6409c61d67b8...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.62105, 'lng': 77.10485}]",14397,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Stay Royal Bed N Breakfast, Jail Road, Maya En...",IND,...,South West,Delhi,Mayapuri,Maya Enclave,110064,28.62092,77.10485,[{'phone': [{'value': '+919212243611'}]}],Jail Road,NaN
9,Apartment Rental-Flat Rental,here:pds:place:356ttngq-54752f7e21024a9cb4b8d7...,en,here:cm:ontology:apartment_rental_flat_rental,place,"[{'lat': 28.70364, 'lng': 77.26037}]",15031,"[{'id': '700-7200-0324', 'name': 'Apartment/Fl...","Apartment Rental-Flat Rental, Mangal Pandey Ma.

In [ ]:
name_df

,title,position.lat,position.lng
0,Kanishka Apartments,28.72978,77.15075
1,Tower Apartment,28.69680,77.14268
2,Baba Guest House,28.71351,77.17497
3,Ganesh Apartment,28.74384,77.20382
4,Narang Boys Pg,28.69576,77.20406
5,Kohinoor Guest House-Vikaspuri,28.63851,77.07318
6,Rooms on Rent,28.64136,77.18549
7,Sadar Guest House,28.65444,77.21121
8,Stay Royal Bed N Breakfast,28.62092,77.10485
9,Apartment Rental-Flat Rental,28.70350,77.26036
